In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, SGDRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_validate
from sklearn.neural_network import MLPRegressor
import warnings
# warnings.filterwarnings("ignore")